In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Model Deployment through Cloud Build

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/06_model_training_pipeline.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/06_model_training_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/06_model_training_pipeline.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.


### Objective

This notebook shows how to use Cloud Build to deploy a model generated by execution of Vertex AI Pipeline. 

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)

Steps performed in this notebook:

    * Deployment of Model triggered on-demand (from within notebook)

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "dryrun01-fraudfinder"
PROJECT              = "dryrun01"
REGION               = "us-central1"
ID                   = "9selj"
FEATURESTORE_ID      = "fraudfinder_9selj"
MODEL_NAME           = "fraudfinder_logreg_model"
ENDPOINT_NAME        = "fraudfinder_logreg_endpoint"
TRAINING_DS_SIZE     = "1000"



In [2]:
PIPELINE_NAME = f'fraud-finder-xgb-pipeline-{ID}'
PIPELINE_STORE_URI = f"gs://{BUCKET_NAME}/pipeline-store/"
IMAGE_REPOSITORY = f"fraudfinder-{ID}"
ENDPOINT_DISPLAY_NAME = f"fraudfinder_xgb_model_frmlz-{ID}_endpoint"
MODEL_DISPLAY_NAME = f"fraudfinder_xgb_model_frmlz-{ID}"

### A) Deployment of Model triggered on-demand (from within notebook)

The cloud build will deploy the latest model. We set here some parameters for the Cloud Build pipeline.

In [3]:
SUBSTITUTIONS=f"""\
_REGION={REGION},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_WORKSPACE_DIR=./
"""

!echo $SUBSTITUTIONS

_REGION=us-central1,_ENDPOINT_DISPLAY_NAME=fraudfinder_xgb_model_frmlz-9selj_endpoint,_MODEL_DISPLAY_NAME=fraudfinder_xgb_model_frmlz-9selj,_WORKSPACE_DIR=./


In [4]:
!gcloud builds submit /home/jupyter/fraudfinder/vertex_ai/ \
--timeout=306m \
--machine-type=e2-highcpu-8 \
--config=deploy_model/modeldeployment.yaml \
--substitutions= --substitutions {SUBSTITUTIONS}

Creating temporary tarball archive of 20 file(s) totalling 1.3 MiB before compression.
Uploading tarball of [/home/jupyter/fraudfinder/vertex_ai/] to [gs://dryrun01_cloudbuild/source/1675417801.779625-f3aa6a9f32c8403587c95c39db9b52dd.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/dryrun01/locations/global/builds/09b98e0b-4a11-43a1-a33e-4b504603d8f0].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/09b98e0b-4a11-43a1-a33e-4b504603d8f0?project=547090333450 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "09b98e0b-4a11-43a1-a33e-4b504603d8f0"

FETCHSOURCE
Fetching storage object: gs://dryrun01_cloudbuild/source/1675417801.779625-f3aa6a9f32c8403587c95c39db9b52dd.tgz#1675417803379671
Copying gs://dryrun01_cloudbuild/source/1675417801.779625-f3aa6a9f32c8403587c95c39db9b52dd.tgz#1675417803379671...
/ [1 files][646.4 KiB/646.4 KiB]                                                
Operation completed over

#### You can now go to endpoints in Vertex AI tab and check for the deployed model. 